In [1]:
# Install key libraries (run this first)
!pip install transformers datasets scikit-learn --quiet
print("✓ Libraries installed!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda

In [2]:
from datasets import load_dataset
import pandas as pd

# Load Twitter data
dataset = load_dataset("sentiment140", split="train").shuffle().select(range(500))
df = dataset.to_pandas()

# Preview a messy tweet
print("Sample tweet with emojis:\n", df.iloc[15]["text"])  # Random index

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.84k [00:00<?, ?B/s]

sentiment140.py:   0%|          | 0.00/4.03k [00:00<?, ?B/s]

The repository for sentiment140 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/sentiment140.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/1600000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/498 [00:00<?, ? examples/s]

Sample tweet with emojis:
 its fathers day when my daddy is in canada  http://bit.ly/2PuZz1


In [3]:
from transformers import pipeline

# BERT (your familiar model)
bert = pipeline("text-classification",
               model="nlptown/bert-base-multilingual-uncased-sentiment")

# XLM-Roberta (new challenger)
xlm = pipeline("text-classification",
              model="cardiffnlp/twitter-xlm-roberta-base-sentiment")

print("✓ Models loaded! Let's compare them...")

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cpu


✓ Models loaded! Let's compare them...


In [7]:
from sklearn.metrics import classification_report
import numpy as np

# Analyze 100 tweets
sample_texts = df.head(100)["text"].tolist()
true_labels = df.head(100)["sentiment"].values  # Original labels (0=negative, 4=positive)

# Convert true labels to match each model's format
# For BERT: Map 0→1★, 4→5★ (5 classes)
true_labels_bert = (true_labels + 1).astype(int)

# For XLM: Map 0→0 (negative), 2→1 (neutral), 4→2 (positive) (3 classes)
true_labels_xlm = np.where(true_labels < 2, 0, np.where(true_labels == 2, 1, 2))

# Get predictions
print("Running BERT...")
bert_preds = [int(bert(text[:512])[0]["label"].replace(" stars", "").replace(" star", ""))
              for text in sample_texts]

print("\nRunning XLM-Roberta...")
xlm_preds = []
for text in sample_texts:
    pred = xlm(text[:512])[0]["label"].lower()
    # Map XLM outputs to numeric labels
    if "negative" in pred:
        xlm_preds.append(0)
    elif "neutral" in pred:
        xlm_preds.append(1)
    else:  # positive
        xlm_preds.append(2)

# Compare BERT (5-class)
print("\nBERT Performance (1-5★ scale):")
print(classification_report(true_labels_bert, bert_preds,
      target_names=["1★", "2★", "3★", "4★", "5★"],
      zero_division=0))  # Silence warnings

# Compare XLM (3-class)
print("\nXLM-Roberta Performance (Negative/Neutral/Positive):")
print(classification_report(true_labels_xlm, xlm_preds,
      target_names=["Negative", "Neutral", "Positive"],
      zero_division=0))

Running BERT...

Running XLM-Roberta...

BERT Performance (1-5★ scale):
              precision    recall  f1-score   support

          1★       0.80      0.53      0.64        53
          2★       0.00      0.00      0.00         0
          3★       0.00      0.00      0.00         0
          4★       0.00      0.00      0.00         0
          5★       0.72      0.49      0.58        47

    accuracy                           0.51       100
   macro avg       0.30      0.20      0.24       100
weighted avg       0.76      0.51      0.61       100


XLM-Roberta Performance (Negative/Neutral/Positive):
              precision    recall  f1-score   support

    Negative       0.80      0.62      0.70        53
     Neutral       0.00      0.00      0.00         0
    Positive       0.72      0.62      0.67        47

    accuracy                           0.62       100
   macro avg       0.51      0.41      0.46       100
weighted avg       0.77      0.62      0.69       100



In [8]:
# Check data structure
print("Columns:", df.columns.tolist())
print("\nSentiment value counts:")
print(df["sentiment"].value_counts())

# Check for missing values
print("\nMissing values per column:")
print(df.isnull().sum())

# Sample 3 tweets per sentiment
print("\nSample tweets:")
for sentiment in sorted(df["sentiment"].unique()):
    samples = df[df["sentiment"] == sentiment].sample(3)["text"].tolist()
    print(f"\nSentiment {sentiment}:")
    for i, tweet in enumerate(samples, 1):
        print(f"{i}. {tweet[:80]}...")

Columns: ['text', 'date', 'user', 'sentiment', 'query']

Sentiment value counts:
sentiment
4    264
0    236
Name: count, dtype: int64

Missing values per column:
text         0
date         0
user         0
sentiment    0
query        0
dtype: int64

Sample tweets:

Sentiment 0:
1. Went to Oak Park Fireworks lastnight. Branden was too young to go  mi baby...
2. in bed still, it's so bright out. i have nothing to eat  i want mallory to be he...
3. Auditions thurs &amp; sat already. Sadly, don't pay ...

Sentiment 4:
1. @Dementia_PT noas ...
2. Last exam over, never have to wear that uniform again  could be interesting how ...
3. @MissAdraPage Cool! that's a good movie! especially when Hugh gets out of the ta...


In [13]:
# Save comparison results
results = pd.DataFrame({
    "Model": ["BERT", "XLM-Roberta"],
    "Accuracy": [0.82, 0.85],  # Replace with your actual metrics
    "F1-Score": [0.81, 0.84]
})
results.to_csv("model_comparison_results.csv", index=False)

# Download
from google.colab import files
files.download("model_comparison_results.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>